<a href="https://colab.research.google.com/github/zolfaShefreie/data-mining-classification/blob/main/pro2_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel('./dataset.xls', parse_dates=True)

In [ ]:
df.columns = ['Index', 'Customer', 'Agent', 'SalesAgentEmailID',
       'ContactEmailID', 'Stage', 'Product', 'Close_Value', 'Created Date',
       'Close Date']

In [ ]:
df.isnull().sum()

Index                   0
Customer                0
Agent                   0
SalesAgentEmailID       0
ContactEmailID          0
Stage                   0
Product                 0
Close_Value          1208
Created Date            0
Close Date              0
dtype: int64

In [ ]:
df['Created Date Year'] =pd.DatetimeIndex(df['Created Date']).year

In [ ]:
df['Close Date Year'] =pd.DatetimeIndex(df['Close Date']).year

In [ ]:
df['days'] = (df['Close Date'] - df['Created Date'])/np.timedelta64(1,'D')

In [ ]:
len(df['Agent'].unique()), len(df['SalesAgentEmailID'].unique())

(30, 30)

In [ ]:
df.drop('Created Date', axis='columns', inplace=True)

In [ ]:
df.drop('Close Date', axis='columns', inplace=True)

In [ ]:
df.drop('Customer', axis='columns', inplace=True)
df.drop('SalesAgentEmailID', axis='columns', inplace=True)
df.drop('ContactEmailID', axis='columns', inplace=True)

In [ ]:
df.columns

Index(['Index', 'Agent', 'Stage', 'Product', 'Close_Value',
       'Created Date Year', 'Close Date Year', 'days'],
      dtype='object')

In [ ]:
group_product_stage = df.groupby(['Product', 'Stage'])['Close_Value'].mean().reset_index()
group_product_stage.columns = ['Product', 'Stage', 'Mean_Close_Value']

In [ ]:
merged_pfs = pd.merge(df, group_product_stage, on=['Product', 'Stage'])
merged_pfs.sort_values(by=['Index'], inplace=True)
merged_pfs.reset_index(inplace=True)
merged_pfs

,index,Index,Agent,Stage,Product,Close_Value,Created Date Year,Close Date Year,days,Mean_Close_Value
0,0,0,Corliss Cosme,Won,GTX Plus Basic,1255.0,2016,2016,141.0,1082.183432
1,590,1,Rosalina Dieter,Won,MG Special,45.0,2016,2016,303.0,55.178322
2,1270,2,Donn Cantrell,Lost,MG Special,NaN,2016,2016,336.0,1260.413793
3,1626,3,James Ascencio,In Progress,MG Advanced,1054.0,2016,2016,249.0,1510.356643
4,1954,4,Lajuana Vencill,Won,MG Advanced,3180.0,2016,2016,24.0,3382.754065
...,...,...,...,...,...,...,...,...,...,...
7795,7416,7795,Jonathan Berthelot,In Progress,GTX Basic,590.0,2018,2019,204.0,1326.275862
7796,1953,7796,Marty Freudenburg,In Progress,MG Advanced,3284.0,2018,2019,147.0,1510.356643
7797,6986,7797,Anna Snelling,In Progress,GTX Plus Pro,300.0,2018,2019,105.0,1344.198953
7798,6216,7798,Gladys Colclough,In Progress,MG Special,3284.0,2018,2019,136.0,1333.502732


In [ ]:
df['Close_Value'].fillna(merged_pfs['Mean_Close_Value'], inplace=True)
df

,Index,Agent,Stage,Product,Close_Value,Created Date Year,Close Date Year,days
0,0,Corliss Cosme,Won,GTX Plus Basic,1255.000000,2016,2016,141.0
1,1,Rosalina Dieter,Won,MG Special,45.000000,2016,2016,303.0
2,2,Donn Cantrell,Lost,MG Special,1260.413793,2016,2016,336.0
3,3,James Ascencio,In Progress,MG Advanced,1054.000000,2016,2016,249.0
4,4,Lajuana Vencill,Won,MG Advanced,3180.000000,2016,2016,24.0
...,...,...,...,...,...,...,...,...
7795,7795,Jonathan Berthelot,In Progress,GTX Basic,590.000000,2018,2019,204.0
7796,7796,Marty Freudenburg,In Progress,MG Advanced,3284.000000,2018,2019,147.0
7797,7797,Anna Snelling,In Progress,GTX Plus Pro,300.000000,2018,2019,105.0
7798,7798,Gladys Colclough,In Progress,MG Special,3284.000000,2018,2019,136.0


In [ ]:
df.isnull().sum()

Index                0
Agent                0
Stage                0
Product              0
Close_Value          0
Created Date Year    0
Close Date Year      0
days                 0
dtype: int64

In [ ]:
def get_stage(close_value, product, stage, df):
  if stage != "In Progress":
    return stage
  min_value = df[df['Product'] == product][df['Stage'] == "Won"].quantile(.1)['Close_Value']
  max_value = df[df['Product'] == product][df['Stage'] == "Won"].quantile(.9)['Close_Value']
  if min_value < close_value and close_value < max_value:
    return "Won"
  return "Lost"

In [ ]:
df['Stage'] = df.apply(lambda row : get_stage(row['Close_Value'], row['Product'], row['Stage'], df), axis = 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [ ]:
from sklearn import preprocessing
labelEncoder = preprocessing.LabelEncoder()
# df['SalesAgentEmailID'] = labelEncoder.fit_transform(df['SalesAgentEmailID'])
# df['ContactEmailID'] = labelEncoder.fit_transform(df['ContactEmailID'])
df['Stage'] = labelEncoder.fit_transform(df['Stage'])
# df['Product'] = labelEncoder.fit_transform(df['Product'])
# df['Created Date Year'] = labelEncoder.fit_transform(df['Created Date Year'])
# df['Close Date Year'] = labelEncoder.fit_transform(df['Close Date Year'])
# df['Close_Value'] = df['Close_Value'] * 100
df

,Index,Agent,Stage,Product,Close_Value,Created Date Year,Close Date Year,days
0,0,Corliss Cosme,1,GTX Plus Basic,1255.000000,2016,2016,141.0
1,1,Rosalina Dieter,1,MG Special,45.000000,2016,2016,303.0
2,2,Donn Cantrell,0,MG Special,1260.413793,2016,2016,336.0
3,3,James Ascencio,0,MG Advanced,1054.000000,2016,2016,249.0
4,4,Lajuana Vencill,1,MG Advanced,3180.000000,2016,2016,24.0
...,...,...,...,...,...,...,...,...
7795,7795,Jonathan Berthelot,1,GTX Basic,590.000000,2018,2019,204.0
7796,7796,Marty Freudenburg,1,MG Advanced,3284.000000,2018,2019,147.0
7797,7797,Anna Snelling,0,GTX Plus Pro,300.000000,2018,2019,105.0
7798,7798,Gladys Colclough,0,MG Special,3284.000000,2018,2019,136.0


In [ ]:
# one_to_hot = preprocessing.OneHotEncoder(handle_unknown='ignore')
# product_df = pd.DataFrame(one_to_hot.fit_transform(df[['Product']]).toarray())
# #2: GTX Plus Basic, 6:MG Special, 5:MG Advanced, 3: GTX Plus Pro, 1:GTX Basic, 0:GTK 500, 4:GTXPro
# product_df.columns = ['GTK 500', 'GTX Basic', 'GTX Plus Basic', 'GTX Plus Pro', 
#                       'GTXPro', 'MG Advanced', 'MG Special']
# df = df.join(product_df)
# df.drop('Product', axis='columns', inplace=True)

In [ ]:
# from sklearn.feature_extraction import FeatureHasher
# hashEncoder = FeatureHasher(n_features=6, input_type='string')
# agent_df = pd.DataFrame(hashEncoder.transform(df['SalesAgentEmailID']).toarray())
# agent_df.columns = ['agent_0', 'agent_1', 'agent_2', 'agent_3', 'agent_4', 'agent_5']
# df = df.join(agent_df)
# df.drop('SalesAgentEmailID', axis='columns', inplace=True)

In [ ]:

choices = list()
close_years = list(df['Close Date Year'].unique())
print(len(close_years))
create_years = list(df['Created Date Year'].unique())
print(len(create_years))
stages = list(df['Stage'].unique())
print(len(stages))
agents = list(df['Agent'].unique())
print(len(agents))
products = list(df['Product'].unique())
print(len(products))
remaind = pd.DataFrame()
for product in products:
  pro_df = df[df['Product']==product]
  for stage in stages:
    pro_st_df = pro_df[pro_df['Stage']==stage]
    for create_year in create_years:
      p_st_c_df = pro_st_df[pro_st_df['Created Date Year']==create_year]
      for close_year in close_years:
        p_s_c2_df = p_st_c_df[p_st_c_df['Close Date Year']==close_year]
        for agent in agents:
          a_p_s_c2_df = p_s_c2_df[p_s_c2_df['Agent']==agent]
          d = pd.DataFrame()
          d = pd.concat([d, a_p_s_c2_df])
          if len(d) > 1 :
            choices.append(d)
          elif len(d) == 1:
            remaind = pd.concat([d, remaind])
choices.append(remaind)
len(choices)

4
3
2
30
7


1093

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
val_x_data = pd.DataFrame()
val_y_data = list()
train_x_data = pd.DataFrame()
train_y_data = list()

In [ ]:
for choice in choices:
  x = choice.drop('Stage', axis=1)
  x = x.drop('Index', axis=1)
  y=choice["Stage"]
  x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    train_size=0.8, 
                                                    random_state=42)
  train_x_data=train_x_data.append(x_train)
  train_y_data+=(y_train.tolist())
  val_x_data=val_x_data.append(x_test)
  val_y_data+=y_test.tolist()

In [ ]:
train_x_data.shape, len(train_y_data), val_x_data.shape, len(val_y_data)

((5791, 6), 5791, (2009, 6), 2009)

In [ ]:
train_x_data.reset_index(inplace=True)
val_x_data.reset_index(inplace=True)

In [ ]:
for var in ['Product', 'Created Date Year', 'Close Date Year', 'Agent']: 
    
    print(val_x_data[var].value_counts())
# print(")))))))))))))))))))))))))))))))))))))))))))))))))")
# print(train_y_data)
# print(train_x_data)

GTX Basic         414
MG Special        373
GTXPro            329
MG Advanced       326
GTX Plus Basic    319
GTX Plus Pro      235
GTK 500            13
Name: Product, dtype: int64
2017    1761
2018     144
2016     104
Name: Created Date Year, dtype: int64
2017    754
2019    605
2018    564
2016     86
Name: Close Date Year, dtype: int64
Darcel Schlecht       154
Anna Snelling          99
Vicki Laflamme         96
Kary Hendrixson        94
Versie Hillebrand      84
                     ... 
Elease Gluck           43
Rosie Papadopoulos     41
Rosalina Dieter        40
Wilburn Farren         34
Garret Kinder          29
Name: Agent, Length: 30, dtype: int64


In [ ]:
print(train_x_data.count())
print(val_x_data.count())
print(df.count())

index                5791
Agent                5791
Product              5791
Close_Value          5791
Created Date Year    5791
Close Date Year      5791
days                 5791
dtype: int64
index                2009
Agent                2009
Product              2009
Close_Value          2009
Created Date Year    2009
Close Date Year      2009
days                 2009
dtype: int64
Index                7800
Agent                7800
Stage                7800
Product              7800
Close_Value          7800
Created Date Year    7800
Close Date Year      7800
days                 7800
dtype: int64


In [ ]:
df['Product'].unique().reshape(-1,1)
# train_x_data.drop('SalesAgentEmailID', axis='columns', inplace=True)
# val_x_data.drop('SalesAgentEmailID', axis='columns', inplace=True)

array([['GTX Plus Basic'],
       ['MG Special'],
       ['MG Advanced'],
       ['GTX Basic'],
       ['GTXPro'],
       ['GTX Plus Pro'],
       ['GTK 500']], dtype=object)

In [ ]:
one_to_hot = preprocessing.OneHotEncoder()
one_to_hot.fit(df['Product'].unique().reshape(-1,1))
print(one_to_hot.get_feature_names())
product_df = pd.DataFrame(one_to_hot.transform(train_x_data[['Product']]).toarray())

#2: GTX Plus Basic, 6:MG Special, 5:MG Advanced, 3: GTX Plus Pro, 1:GTX Basic, 0:GTK 500, 4:GTXPro
product_df.columns = ['GTK 500', 'GTX Basic', 'GTX Plus Basic', 'GTX Plus Pro', 
                      'GTXPro', 'MG Advanced', 'MG Special']

train_x_data = train_x_data.join(product_df)
train_x_data.drop('Product', axis='columns', inplace=True)

################################################
product_df = pd.DataFrame(one_to_hot.transform(val_x_data[['Product']]).toarray())
#2: GTX Plus Basic, 6:MG Special, 5:MG Advanced, 3: GTX Plus Pro, 1:GTX Basic, 0:GTK 500, 4:GTXPro
product_df.columns = ['GTK 500', 'GTX Basic', 'GTX Plus Basic', 'GTX Plus Pro', 
                      'GTXPro', 'MG Advanced', 'MG Special']
val_x_data = val_x_data.join(product_df)
val_x_data.drop('Product', axis='columns', inplace=True)
train_x_data

['x0_GTK 500' 'x0_GTX Basic' 'x0_GTX Plus Basic' 'x0_GTX Plus Pro'
 'x0_GTXPro' 'x0_MG Advanced' 'x0_MG Special']


,index,Agent,Close_Value,Created Date Year,Close Date Year,days,GTK 500,GTX Basic,GTX Plus Basic,GTX Plus Pro,GTXPro,MG Advanced,MG Special
0,0,Corliss Cosme,1255.0,2016,2016,141.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,64,Kary Hendrixson,988.0,2016,2016,65.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,78,Darcel Schlecht,1161.0,2016,2016,141.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,101,Daniell Hammack,1037.0,2016,2016,38.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,204,Hayden Neloms,1108.0,2016,2016,59.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5786,7440,Niesha Huffines,493.0,2018,2019,537.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5787,2996,Hayden Neloms,560.0,2017,2019,764.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5788,2802,Versie Hillebrand,1037.0,2017,2019,943.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5789,222,Markita Hansen,61.0,2016,2016,114.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
pd.set_option('display.max_rows', 10)
product_df

,GTK 500,GTX Basic,GTX Plus Basic,GTX Plus Pro,GTXPro,MG Advanced,MG Special
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2004,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2005,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2006,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2007,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
from sklearn.feature_extraction import FeatureHasher
hashEncoder = FeatureHasher(n_features=6, input_type='string')
hashEncoder.fit(list(df['Agent'].unique()))

agent_df = pd.DataFrame(hashEncoder.transform(train_x_data['Agent']).toarray())
agent_df.columns = ['agent_0', 'agent_1', 'agent_2', 'agent_3', 'agent_4', 'agent_5']
train_x_data = train_x_data.join(agent_df)
train_x_data.drop('Agent', axis='columns', inplace=True)


#########################3
agent_df = pd.DataFrame(hashEncoder.transform(val_x_data['Agent']).toarray())
agent_df.columns = ['agent_0', 'agent_1', 'agent_2', 'agent_3', 'agent_4', 'agent_5']
val_x_data = val_x_data.join(agent_df)
val_x_data.drop('Agent', axis='columns', inplace=True)


In [ ]:
# x = df.drop('Stage', axis=1)
# x = x.drop('Index', axis=1)
# y = df["Stage"]
# x.shape

In [ ]:
# from sklearn.model_selection import train_test_split

# x_train, x_test, y_train, y_test = train_test_split(x, y, 
#                                                     train_size=0.8, 
#                                                     random_state=42)

In [ ]:
# x_train.shape

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [ ]:
train_x_data

,index,Close_Value,Created Date Year,Close Date Year,days,GTK 500,GTX Basic,GTX Plus Basic,GTX Plus Pro,GTXPro,MG Advanced,MG Special,agent_0,agent_1,agent_2,agent_3,agent_4,agent_5
0,0,1255.0,2016,2016,141.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,-6.0,1.0,0.0
1,64,988.0,2016,2016,65.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-2.0,6.0,2.0,-2.0,0.0,1.0
2,78,1161.0,2016,2016,141.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,3.0,5.0,-1.0,0.0,-5.0
3,101,1037.0,2016,2016,38.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,2.0,7.0,-1.0,2.0,-2.0
4,204,1108.0,2016,2016,59.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,5.0,3.0,-1.0,2.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5786,7440,493.0,2018,2019,537.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,3.0,2.0,-4.0,1.0,0.0
5787,2996,560.0,2017,2019,764.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,5.0,3.0,-1.0,2.0,-1.0
5788,2802,1037.0,2017,2019,943.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,7.0,4.0,-2.0,0.0,-1.0
5789,222,61.0,2016,2016,114.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-2.0,3.0,4.0,-3.0,1.0,-1.0


In [ ]:

df.iloc[7440]

Index                           7440
Agent                Niesha Huffines
Stage                              1
Product                    GTX Basic
Close_Value                      493
Created Date Year               2018
Close Date Year                 2019
days                             537
Name: 7440, dtype: object

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(train_x_data, train_y_data)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
y_pred = clf.predict(val_x_data)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(val_y_data, y_pred))
print(classification_report(val_y_data, y_pred))

[[928  63]
 [ 71 947]]
              precision    recall  f1-score   support

           0       0.93      0.94      0.93       991
           1       0.94      0.93      0.93      1018

    accuracy                           0.93      2009
   macro avg       0.93      0.93      0.93      2009
weighted avg       0.93      0.93      0.93      2009



In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(random_state=30, n_estimators=100, criterion='entropy', max_depth=17, bootstrap=False)
clf.fit(train_x_data, train_y_data)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=17, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=30, verbose=0,
                       warm_start=False)

In [ ]:
y_pred2 = clf.predict(val_x_data)

In [ ]:
print(confusion_matrix(val_y_data, y_pred2))
print(classification_report(val_y_data, y_pred2))

[[889 102]
 [ 38 980]]
              precision    recall  f1-score   support

           0       0.96      0.90      0.93       991
           1       0.91      0.96      0.93      1018

    accuracy                           0.93      2009
   macro avg       0.93      0.93      0.93      2009
weighted avg       0.93      0.93      0.93      2009



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(train_x_data, train_y_data)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [ ]:
y_pred3 = neigh.predict(val_x_data)

In [ ]:
print(confusion_matrix(val_y_data, y_pred3))
print(classification_report(val_y_data, y_pred3))

[[712 279]
 [411 607]]
              precision    recall  f1-score   support

           0       0.63      0.72      0.67       991
           1       0.69      0.60      0.64      1018

    accuracy                           0.66      2009
   macro avg       0.66      0.66      0.66      2009
weighted avg       0.66      0.66      0.66      2009



In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred4 = gnb.fit(train_x_data, train_y_data).predict(val_x_data)

In [ ]:
print(confusion_matrix(val_y_data, y_pred4))
print(classification_report(val_y_data, y_pred4))

[[495 496]
 [302 716]]
              precision    recall  f1-score   support

           0       0.62      0.50      0.55       991
           1       0.59      0.70      0.64      1018

    accuracy                           0.60      2009
   macro avg       0.61      0.60      0.60      2009
weighted avg       0.61      0.60      0.60      2009



check the best n_features for hashing agents

In [ ]:
from sklearn.feature_extraction import FeatureHasher
agents = list(df['Agent'].unique()) + ['other']
hashEncoder = FeatureHasher(n_features=6, input_type='string', alternate_sign=False)
l = hashEncoder.fit_transform(agents).toarray()
print(len(l))
# print(list(l))
# print(len(set(list(l))))
s = list()
for each in l:
  if list(each) in s:
    print('here')
  s.append(list(each))
  print(each)
# hashEncoder.transform(['a', 'b', 'c', 'd']).toarray()

31
[0. 4. 2. 6. 1. 0.]
[1. 4. 5. 5. 0. 0.]
[3. 3. 5. 2. 0. 0.]
[1. 4. 2. 3. 2. 2.]
[2. 2. 6. 3. 1. 1.]
[1. 2. 2. 1. 1. 2.]
[2. 6. 2. 2. 2. 1.]
[1. 3. 5. 1. 0. 5.]
[1. 6. 2. 3. 1. 0.]
[0. 2. 4. 3. 1. 2.]
[1. 2. 7. 1. 2. 2.]
[1. 7. 4. 4. 0. 1.]
[5. 1. 5. 1. 1. 0.]
[2. 3. 4. 3. 1. 1.]
[0. 1. 4. 4. 2. 3.]
[0. 3. 6. 3. 2. 2.]
[2. 4. 5. 1. 1. 1.]
[1. 4. 6. 3. 1. 1.]
[0. 4. 3. 3. 1. 0.]
[2. 6. 4. 4. 1. 1.]
[0. 3. 2. 6. 1. 0.]
[1. 5. 3. 1. 2. 1.]
[2. 5. 4. 0. 3. 1.]
[2. 6. 5. 3. 0. 2.]
[1. 1. 5. 2. 2. 2.]
[2. 5. 3. 2. 0. 0.]
[1. 3. 2. 4. 1. 4.]
[1. 6. 4. 1. 4. 1.]
[2. 1. 3. 4. 2. 2.]
[0. 2. 4. 3. 0. 0.]
[0. 3. 0. 1. 0. 1.]
